In [1]:
import numpy as np

In [2]:
from pathlib import Path
import pandas as pd

data_root = Path("../kkdata3")
for x in data_root.glob("*"):
    print(x)

train_target = pd.read_parquet(data_root / "label_train_target.parquet")
test_source = pd.read_parquet(data_root / "label_test_source.parquet")
meta_song = pd.read_parquet(data_root / "meta_song.parquet")

../kkdata3/predict_result.parquet
../kkdata3/label_train_source.parquet
../kkdata3/label_train_target.parquet
../kkdata3/label_test_source.parquet
../kkdata3/meta_song_titletext.parquet
../kkdata3/sample.csv
../kkdata3/meta_song_lyricist.parquet
../kkdata3/meta_song_producer.parquet
../kkdata3/meta_song.parquet
../kkdata3/meta_song_genre.parquet
../kkdata3/meta_song_composer.parquet


In [3]:
train_source = pd.read_parquet(data_root / "label_train_source.parquet")
train_source.sort_values(["session_id", "listening_order"], inplace=True)
train_target.sort_values(["session_id", "listening_order"], inplace=True)
# test_source.sort_values(["session_id", "listening_order"], inplace=True)

In [4]:
# return n+1 column song id
def getTrainData(df, n=2):
    df = df.copy()
    # gen n song id be the dataset
    for i in range(1, n + 1):
        df[f"next{i}_song_id"] = df["song_id"].shift(-i)

    # check if last song id is in the same session
    df[f"next{n}_session_id"] = df["session_id"].shift(-n)
    df = df.query(f"session_id == next{n}_session_id")

    # only get the song_id and next1_song_id, next2_song_id, next3_song_id... column
    df = df[["song_id"] + [f"next{i}_song_id" for i in range(1, n + 1)]]

    return df

In [5]:
src = getTrainData(train_source, 19)
tgt = getTrainData(train_target, 4)

src.reset_index(drop=True, inplace=True)
tgt.reset_index(drop=True, inplace=True)

src.shape[0] == tgt.shape[0]

True

In [34]:
src[src.duplicated()].sort_values("song_id").sort_index()
tgt[src.duplicated()].sort_values("song_id").sort_index()

,song_id,next1_song_id,next2_song_id,next3_song_id,next4_song_id
217,20e137c89079bade9c596d92757afa59,dfcce1f1ae5fed1c038830ea5f4e559b,e1517d538da361e7dc58f77bf187d00c,0a7be36a7ea8d7bcb6dc705361fac0ea,916bf51f5bce48fecda79154c93e1ce5
590,34af8c382bee0cb49c80aa94a3d832d7,34af8c382bee0cb49c80aa94a3d832d7,34af8c382bee0cb49c80aa94a3d832d7,34af8c382bee0cb49c80aa94a3d832d7,34af8c382bee0cb49c80aa94a3d832d7
617,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d
618,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d
619,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d,04cb62ee0fb4fb641e0e71d385f09a0d
...,...,...,...,...,...
572101,2575f3264b8562f85a068d8c1cb74632,2575f3264b8562f85a068d8c1cb74632,2575f3264b8562f85a068d8c1cb74632,2575f3264b8562f85a068d8c1cb74632,2575f3264b8562f85a068d8c1cb74632
572142,7b4ec012f3f5580e57add7139b1c6e02,8797a41daa53cab47d7aa0d75bd7dbb4,98e159f2fe90193c82209674b638ee1c,7b4ec012f3f5580e57add7139b1c6e02,66e6cbab82630d69acca2115157fbcba
572205,8fb82972d63dc711b8275b9330be82b6,8fb82972d63dc711b8275b9330be82b6,8fb82972d63dc711b8275b9330be82b6,8fb82972d63dc711b8275b9330be82b6,8fb82972d63dc711b8275b9330be82b6
572233,9ad9c63b533186746f04a612abca0860,7e0caaaa8bd33ad2c6e6000458e4b2b1,767730509063e5779182491d1668aac0,9fe2262287d3f0415f6e5c8999ee8a1c,fdef56717e5431c7b1c8325222f2406a


In [35]:
duplicates_mask = src.duplicated()

In [8]:
tgt_filtered = tgt[~duplicates_mask].sort_values("song_id").sort_index()
src_filtered = src[~duplicates_mask].sort_values("song_id").sort_index()
tgt_filtered.shape

(528072, 5)

In [9]:
# show duplicated index
src[src.duplicated(keep=False)].index

Index([   216,    217,    332,    589,    590,    616,    617,    618,    619,
          620,
       ...
       571868, 571869, 571870, 571872, 571971, 572101, 572142, 572205, 572233,
       572251],
      dtype='int64', length=50852)

In [10]:
# calculate ngram frequency.
# n=1 is the most frequent of song_id
# n=2 is the most frequent of [song_id and next1_song_id], get the result from next2_song_id column
def getFreq(df, n=2):
    df_train = getTrainData(df, n)
    # calculate ngram frequency
    df_freq = (
        df_train.groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .value_counts(sort=True, normalize=True)
        .reset_index(name="freq")
    )

    # get the most frequent song_id, sort by freq
    df_freq = (
        df_freq.sort_values(
            ["song_id"] + [f"next{i}_song_id" for i in range(1, n)] + ["freq"],
            ascending=False,
        )
        .groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .head(1)
    )
    return df_freq

In [11]:
# get testX dataset
def getTestX(df, n=2):
    trainX = pd.DataFrame()
    # if n == 3:, get listening_order : 18, 19, 20
    for i in range(n, 0, -1):
        _ = df.query(f"listening_order == {20-i+1}")[
            ["session_id", "song_id"]
        ].set_index("session_id")
        if trainX.empty:
            trainX = _
        else:
            trainX = trainX.join(_, lsuffix=f"_{n-i-1}")

    trainX = trainX.rename(columns={trainX.columns[-1]: f"song_id_{n-1}"})

    # trainX keep session_id
    trainX.reset_index(inplace=True)
    trainX.rename_axis("session_id", axis=1, inplace=True)
    return trainX


testX = getTestX(test_source, n=2)
testX.head(5)

session_id,session_id,song_id_0,song_id_1
0,598,8cd8008ddffd9b2d12b6b7bf17474e9e,e8bbc37dee09b8e548f0e2137dd1c2a5
1,1039,1ac480c8add024f6febd936dac34bf82,308f5f2b339b3ff74c9b60b49832d27f
2,1199,3aa676e51e8958a78394f6b6a07c2cea,d1492ba4a2fdf555510934e9740ea8c3
3,1489,e960bf0e27715aeae5620e6c60dead5f,4c19e9de0a90902ccd3b55374942fc5d
4,1868,337f6da1c3f672d989537305209c5347,4a12c85cadde3aaee76be944624a3516


In [12]:
# predict the next song id by ngram frequency table
def predict(df, freqTable, n=2):
    df = pd.merge(
        df,
        freqTable,
        how="left",
        left_on=[f"song_id_{i}" for i in range(n)],
        right_on=["song_id"] + [f"next{_}_song_id" for _ in range(1, n)],
    )
    for i in range(1, n):
        del df[f"next{i}_song_id"]
    del df["song_id"]
    df.rename(columns={f"next{n}_song_id": f"next_song_id"}, inplace=True)
    df.sort_values("session_id")
    return df

In [13]:
# merge the predicted result and new way freq table
def merge2freq(df, freq_table):
    # offset=2 : session_id, freq, next_song_id
    df_offset = 3
    # freq_offset: next_song_id, freq
    freq_offset = 2

    # Delete the unneed column:freq
    if "freq" in df.columns:
        df = df.drop(columns=["freq"])
        df_offset -= 1
    if "freq" in freq_table.columns:
        freq_table = freq_table.drop(columns=["freq"])
        freq_offset -= 1

    fomr_count = df.shape[1] - df_offset  # 2gram : 5 -3 = 2
    latr_count = freq_table.shape[1] - freq_offset  # 1gram freq: 3 - 2 = 1

    left_on = [f"song_id_{i+(fomr_count-latr_count)}" for i in range(latr_count)]
    right_on = ["song_id"] + [f"next{i+1}_song_id" for i in range(latr_count - 1)]
    print("Left on:", left_on)
    print("Right on:", right_on)

    print("r:" + f"song_id_{i}" for i in range(latr_count))
    merged = pd.merge(
        df,
        freq_table,
        how="left",
        left_on=[f"song_id_{i+(fomr_count-latr_count)}" for i in range(latr_count)],
        right_on=["song_id"] + [f"next{i+1}_song_id" for i in range(latr_count - 1)],
    )

    freq_target_name = f"next{latr_count}_song_id"
    merged["next_song_id"].fillna(merged[freq_target_name], inplace=True)

    merged = merged.drop(columns=right_on + [freq_target_name])

    return merged

# experiments

In [14]:
# map song_id to song_index and set song_idx to song to save memory and speed up
meta_song["song_index"] = meta_song.index
train_source = train_source.merge(
    meta_song[["song_id", "song_index"]], on="song_id", how="left"
)
train_target = train_target.merge(
    meta_song[["song_id", "song_index"]], on="song_id", how="left"
)
test_source = test_source.merge(
    meta_song[["song_id", "song_index"]], on="song_id", how="left"
)

train_source["song_id"] = train_source["song_index"]
train_target["song_id"] = train_target["song_index"]
test_source["song_id"] = test_source["song_index"]

del train_source["song_index"]
del train_target["song_index"]
del test_source["song_index"]

In [15]:
trainX = pd.concat([train_source, test_source], axis=0, ignore_index=True)

In [16]:
n = 1
testX = getTestX(test_source, n)
freq1 = getFreq(trainX, n)
freq1
result1 = predict(testX, freq1, n)
result1.head()

,session_id,song_id_0,next_song_id,freq
0,598,320485,320485.0,0.049930
1,1039,82103,245916.0,0.357143
2,1199,82260,82260.0,0.097738
3,1489,163516,58880.0,0.500000
4,1868,736058,798178.0,0.069952


In [17]:
result1.sort_values(["song_id_0", "session_id"], inplace=True)
result1

,session_id,song_id_0,next_song_id,freq
64449,294268,21,21.0,0.049062
139062,301108,21,21.0,0.049062
117995,390969,21,21.0,0.049062
18743,541198,21,21.0,0.049062
25912,116369,39,270774.0,0.165517
...,...,...,...,...
53933,544689,1030708,736280.0,0.170886
135082,577739,1030708,736280.0,0.170886
72807,624199,1030708,736280.0,0.170886
112178,240878,1030709,915034.0,0.046632


In [18]:
n = 2
testX = getTestX(test_source, n)
freq2 = getFreq(trainX, n)
result2 = predict(testX, freq2, n)
freq2

,song_id,next1_song_id,next2_song_id,freq
10095227,1030711,967508.0,561432.0,1.0
10095226,1030711,967484.0,609494.0,1.0
10095225,1030711,946599.0,125355.0,1.0
10095224,1030711,936567.0,1023129.0,1.0
10095223,1030711,643112.0,6835.0,1.0
...,...,...,...,...
4,3,401307.0,318166.0,1.0
3,3,200709.0,1021838.0,1.0
2,3,87846.0,897082.0,1.0
1,1,1014814.0,799780.0,1.0


In [19]:
freq2.sort_values(["song_id", "next1_song_id"], inplace=True)
freq2

,song_id,next1_song_id,next2_song_id,freq
0,1,488417.0,697989.0,1.0
1,1,1014814.0,799780.0,1.0
2,3,87846.0,897082.0,1.0
3,3,200709.0,1021838.0,1.0
4,3,401307.0,318166.0,1.0
...,...,...,...,...
10095223,1030711,643112.0,6835.0,1.0
10095224,1030711,936567.0,1023129.0,1.0
10095225,1030711,946599.0,125355.0,1.0
10095226,1030711,967484.0,609494.0,1.0


In [20]:
n = 2
testX = getTestX(test_source, n)
freq2 = getFreq(trainX, n)
result2 = predict(testX, freq2, n)
# result2.head()

# 2 1
result2_1 = merge2freq(result2, freq1)
result2_1.head()

Left on: ['song_id_1']
Right on: ['song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7f5b7be1b060>


,session_id,song_id_0,song_id_1,next_song_id
0,598,487545,320485,801419.0
1,1039,388680,82103,245916.0
2,1199,187475,82260,82260.0
3,1489,672342,163516,58880.0
4,1868,798178,736058,1001782.0


In [21]:
n = 3
testX = getTestX(test_source, n)
freq3 = getFreq(trainX, n)
result3 = predict(testX, freq3, n)
# result3.head()


# 3 2
result3_2 = merge2freq(result3, freq2)
# [3,2] 1
result3_2_1 = merge2freq(result3_2, freq1)
# [3,2,1] 20
result3_2_1["next_song_id"].fillna(result3_2_1["song_id_2"], inplace=True)
result = result3_2_1
result.head()

Left on: ['song_id_1', 'song_id_2']
Right on: ['song_id', 'next1_song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7f5b7be1b6f0>
Left on: ['song_id_1']
Right on: ['song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7f5b7be1b6f0>


,session_id,song_id_0,song_id_1,song_id_2,next_song_id
0,598,309466,487545,320485,801419.0
1,1039,810606,388680,82103,388680.0
2,1199,524289,187475,82260,419795.0
3,1489,674238,672342,163516,705778.0
4,1868,935208,798178,736058,710126.0


In [22]:
n = 5
testX = getTestX(test_source, n)
freq5 = getFreq(trainX, n)

In [23]:
# calculate ngram frequency.
# n=1 is the most frequent of song_id
# n=2 is the most frequent of [song_id and next1_song_id], get the result from next2_song_id column
def getFreq2(df, n=2, predict=1, threshold_value=0):
    # get df's n+predict numbers of song id
    df_train = getTrainData(df, n + predict - 1)
    # calculate ngram frequency
    df_freq = (
        df_train.groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .value_counts(sort=True, normalize=True)
        .reset_index(name="freq")
    )

    # get the most frequent song_id, sort by freq
    df_freq = (
        df_freq.sort_values(
            ["song_id"] + [f"next{i}_song_id" for i in range(1, n)] + ["freq"],
            ascending=False,
        )
        .groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .head(1)
    )

    df_freq = df_freq[df_freq["freq"] > threshold_value]
    return df_freq


# predict the next song id by ngram frequency table
def predict2(df, freqTable, n=2, target=2):
    df = pd.merge(
        df,
        freqTable,
        how="left",
        left_on=[f"song_id_{i}" for i in range(n)],
        right_on=["song_id"] + [f"next{_}_song_id" for _ in range(1, n)],
    )
    for i in range(1, n):
        del df[f"next{i}_song_id"]
    del df["song_id"]
    for i in range(target):
        df.rename(columns={f"next{n+i}_song_id": f"top{i+1}"}, inplace=True)
    df.sort_values("session_id")
    return df

In [24]:
# 5gen5
n = 5
target = 5
testX = getTestX(test_source, n)
freq5_5 = getFreq2(trainX, n, target, 0.4)
result5_5 = predict2(testX, freq5_5, n, target)
result_5 = result5_5

# 4gen5
col_name = [
    "session_id",
    "song_id_0",
    "song_id_1",
    "song_id_2",
    "song_id_3",
]
testX = testX.drop(columns="song_id_0")
testX.columns = col_name

n = 4
target = 5
freq3_5 = getFreq2(trainX, n, target, 0.4)
result3_5 = predict2(testX, freq3_5, n, target)
result_5 = result_5.fillna(result3_5)


# 3gen5
col_name = [
    "session_id",
    "song_id_0",
    "song_id_1",
    "song_id_2",
]
testX = testX.drop(columns="song_id_0")
testX.columns = col_name

n = 3
target = 5
freq3_5 = getFreq2(trainX, n, target, 0.4)
result3_5 = predict2(testX, freq3_5, n, target)
result_5 = result_5.fillna(result3_5)


result_5.head()

,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,NaN,NaN,NaN,NaN,NaN,NaN
1,1039,229992,252014,810606,388680,82103,NaN,NaN,NaN,NaN,NaN,NaN
2,1199,524289,976710,524289,187475,82260,NaN,NaN,NaN,NaN,NaN,NaN
3,1489,847611,642783,674238,672342,163516,NaN,NaN,NaN,NaN,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.6


In [30]:
# non empty rate
print(1 - (result_5["freq"].isna().sum() / result_5.shape[0]))

result_5["freq"].describe()

0.22023010680534583


count    31507.000000
mean         0.917154
std          0.163295
min          0.402878
25%          0.965432
50%          1.000000
75%          1.000000
max          1.000000
Name: freq, dtype: float64

In [26]:
len(result_5)

143064

In [27]:
31507 / 143064

0.22023010680534585

In [31]:
n = 5
target = 4
testX = getTestX(test_source, n)
freq = getFreq2(trainX, n, target, 0.3)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)


result_5.head()

,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,NaN,NaN,NaN,NaN,NaN,NaN
1,1039,229992,252014,810606,388680,82103,NaN,NaN,NaN,NaN,NaN,NaN
2,1199,524289,976710,524289,187475,82260,NaN,NaN,NaN,NaN,NaN,NaN
3,1489,847611,642783,674238,672342,163516,NaN,NaN,NaN,NaN,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.6


In [32]:
col_name = [
    "session_id",
    "song_id_0",
    "song_id_1",
    "song_id_2",
    "song_id_3",
]
testX = testX.drop(columns="song_id_0")
testX.columns = col_name

n = 4
target = 4
freq = getFreq2(trainX, n, target, 0.3)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)

n = 4
target = 3
freq = getFreq2(trainX, n, target, 0.3)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)

result_5.head()

,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,NaN,NaN,NaN,NaN,NaN,NaN
1,1039,229992,252014,810606,388680,82103,NaN,NaN,NaN,NaN,NaN,NaN
2,1199,524289,976710,524289,187475,82260,NaN,NaN,NaN,NaN,NaN,NaN
3,1489,847611,642783,674238,672342,163516,NaN,NaN,NaN,NaN,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.6


In [33]:
n = 5
target = 3
testX = getTestX(test_source, 5)
freq = getFreq2(trainX, n, target, 0.3)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)


n = 5
target = 2
freq = getFreq2(trainX, n, target, 0.3)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)

n = 5
target = 1
freq = getFreq2(trainX, n, target, 0.1)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)

result_5.head()

,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,NaN,NaN,NaN,NaN,NaN,NaN
1,1039,229992,252014,810606,388680,82103,NaN,NaN,NaN,NaN,NaN,NaN
2,1199,524289,976710,524289,187475,82260,NaN,NaN,NaN,NaN,NaN,NaN
3,1489,847611,642783,674238,672342,163516,NaN,NaN,NaN,NaN,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.6


In [40]:
# 5 gen 5 freq table
freq5_5

,song_id,next1_song_id,next2_song_id,next3_song_id,next4_song_id,next5_song_id,next6_song_id,next7_song_id,next8_song_id,next9_song_id,freq
6856030,1030711,967484.0,609494.0,967484.0,725932.0,934560.0,476351.0,419510.0,199845.0,932886.0,1.0
6856029,1030711,946599.0,125355.0,552899.0,539215.0,561432.0,936567.0,266269.0,837973.0,967508.0,1.0
6856028,1030711,936567.0,1023129.0,571476.0,967508.0,119291.0,1002892.0,967484.0,518949.0,729237.0,1.0
6856027,1030711,643112.0,6835.0,111132.0,552899.0,67869.0,967508.0,174327.0,525414.0,174327.0,1.0
6856026,1030711,642148.0,913721.0,681.0,967484.0,967508.0,571476.0,561432.0,725932.0,643112.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4,3,906384.0,860711.0,44990.0,243158.0,175065.0,243158.0,177925.0,1011604.0,1014667.0,1.0
3,3,401307.0,318166.0,401307.0,107851.0,318166.0,408025.0,318166.0,408025.0,54170.0,1.0
2,3,200709.0,1021838.0,995187.0,964869.0,168421.0,215169.0,265052.0,766339.0,801498.0,1.0
1,3,87846.0,897082.0,813298.0,797523.0,821017.0,769577.0,816094.0,923958.0,420939.0,1.0


In [41]:
result_5

,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,NaN,NaN,NaN,NaN,NaN,NaN
1,1039,229992,252014,810606,388680,82103,NaN,NaN,NaN,NaN,NaN,NaN
2,1199,524289,976710,524289,187475,82260,NaN,NaN,NaN,NaN,NaN,NaN
3,1489,847611,642783,674238,672342,163516,NaN,NaN,NaN,NaN,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535,1028535,1028535.0,1028535.0,1028535.0,1028535.0,1028535.0,0.988636
143060,317578,467984,610999,140463,508806,352697,NaN,NaN,NaN,NaN,NaN,NaN
143061,712209,578489,16604,539300,943003,831678,NaN,NaN,NaN,NaN,NaN,NaN
143062,713779,720557,735314,790777,708757,862975,943720.0,895102.0,964198.0,851907.0,867350.0,0.997268


In [36]:
# non empty rate
print(1 - (result_5.isna().sum() / result_5.shape[0]))

session_id    1.000000
song_id_0     1.000000
song_id_1     1.000000
song_id_2     1.000000
song_id_3     1.000000
song_id_4     1.000000
top1          0.241116
top2          0.237439
top3          0.233972
top4          0.229331
top5          0.220230
freq          0.220230
dtype: float64


In [29]:
# 保留第一個重複值
top15 = result_5[[f"top{i+1}" for i in range(5)]]
top15 = top15.apply(lambda x: x.mask(x.duplicated(keep="first"), np.nan), axis=1)
result_5[["top1", "top2", "top3", "top4", "top5"]] = top15

In [30]:
n = 5
freq5 = getFreq(trainX, n)

# 5 gen 1
n = 5
target = 1
freq = getFreq2(trainX, n, target, 0.1)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)

# cal 5 2 1 gram
testX = predict(testX, freq5, n)
testX = merge2freq(testX, freq2)
testX = merge2freq(testX, freq1)
# testX["next_song_id"].fillna(testX["song_id_4"], inplace=True)

# combine
result_5["top1"] = result_5["top1"].fillna(testX["next_song_id"])

Left on: ['song_id_3', 'song_id_4']
Right on: ['song_id', 'next1_song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf386d0>
Left on: ['song_id_3']
Right on: ['song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf386d0>


In [31]:
col_name = [
    "session_id",
    "song_id_0",
    "song_id_1",
    "song_id_2",
    "song_id_3",
    "song_id_4",
]
testX = testX.drop(columns="song_id_0")
testX.columns = col_name

n = 5
target = 1
freq = getFreq2(trainX, n, target, 0.1)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)

# cal 5 2 1 gram
result5 = predict(testX, freq5, n)
testX = merge2freq(result5, freq2)
testX = merge2freq(testX, freq1)
# testX["next_song_id"].fillna(testX["song_id_4"], inplace=True)

# combine
result_5["top2"] = result_5["top2"].fillna(testX["next_song_id"])
result_5

Left on: ['song_id_3', 'song_id_4']
Right on: ['song_id', 'next1_song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf385f0>
Left on: ['song_id_3']
Right on: ['song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf385f0>


,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,801419.0,946515.0,NaN,NaN,NaN,NaN
1,1039,229992,252014,810606,388680,82103,388680.0,245916.0,NaN,NaN,NaN,NaN
2,1199,524289,976710,524289,187475,82260,419795.0,82260.0,NaN,NaN,NaN,NaN
3,1489,847611,642783,674238,672342,163516,705778.0,58880.0,NaN,NaN,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535,1028535,1028535.0,1028535.0,NaN,NaN,NaN,0.988636
143060,317578,467984,610999,140463,508806,352697,352697.0,NaN,NaN,NaN,NaN,NaN
143061,712209,578489,16604,539300,943003,831678,975708.0,997112.0,NaN,NaN,NaN,NaN
143062,713779,720557,735314,790777,708757,862975,943720.0,895102.0,964198.0,851907.0,867350.0,0.997268


In [32]:
col_name = [
    "session_id",
    "song_id_0",
    "song_id_1",
    "song_id_2",
    "song_id_3",
    "song_id_4",
]
testX = testX.drop(columns="song_id_0")
testX.columns = col_name

n = 5
target = 1
freq = getFreq2(trainX, n, target, 0.1)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)

# cal 5 2 1 gram
result5 = predict(testX, freq5, n)
testX = merge2freq(result5, freq2)
testX = merge2freq(testX, freq1)
testX["next_song_id"].fillna(testX["song_id_4"], inplace=True)

# combine
result_5["top3"] = result_5["top3"].fillna(testX["next_song_id"])
result_5

Left on: ['song_id_3', 'song_id_4']
Right on: ['song_id', 'next1_song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf38c10>
Left on: ['song_id_3']
Right on: ['song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf38c10>


,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,801419.0,946515.0,660101.0,NaN,NaN,NaN
1,1039,229992,252014,810606,388680,82103,388680.0,245916.0,388680.0,NaN,NaN,NaN
2,1199,524289,976710,524289,187475,82260,419795.0,82260.0,634769.0,NaN,NaN,NaN
3,1489,847611,642783,674238,672342,163516,705778.0,58880.0,905285.0,NaN,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535,1028535,1028535.0,1028535.0,1028535.0,NaN,NaN,0.988636
143060,317578,467984,610999,140463,508806,352697,352697.0,NaN,NaN,NaN,NaN,NaN
143061,712209,578489,16604,539300,943003,831678,975708.0,997112.0,788164.0,NaN,NaN,NaN
143062,713779,720557,735314,790777,708757,862975,943720.0,895102.0,964198.0,851907.0,867350.0,0.997268


In [33]:
col_name = [
    "session_id",
    "song_id_0",
    "song_id_1",
    "song_id_2",
    "song_id_3",
    "song_id_4",
]
testX = testX.drop(columns="song_id_0")
testX.columns = col_name

# 5 gen 1
n = 5
target = 1
freq = getFreq2(trainX, n, target, 0.1)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)

# cal 5 2 1 gram
result5 = predict(testX, freq5, n)
testX = merge2freq(result5, freq2)
testX["next_song_id"].fillna(testX["song_id_4"], inplace=True)
testX = merge2freq(testX, freq1)

# combine
result_5["top4"] = result_5["top4"].fillna(testX["next_song_id"])
result_5

Left on: ['song_id_3', 'song_id_4']
Right on: ['song_id', 'next1_song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf38270>
Left on: ['song_id_3']
Right on: ['song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf38270>


,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,801419.0,946515.0,660101.0,413130.0,NaN,NaN
1,1039,229992,252014,810606,388680,82103,388680.0,245916.0,388680.0,388680.0,NaN,NaN
2,1199,524289,976710,524289,187475,82260,419795.0,82260.0,634769.0,634769.0,NaN,NaN
3,1489,847611,642783,674238,672342,163516,705778.0,58880.0,905285.0,905285.0,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535,1028535,1028535.0,1028535.0,1028535.0,1028535.0,NaN,0.988636
143060,317578,467984,610999,140463,508806,352697,352697.0,NaN,NaN,NaN,NaN,NaN
143061,712209,578489,16604,539300,943003,831678,975708.0,997112.0,788164.0,788164.0,NaN,NaN
143062,713779,720557,735314,790777,708757,862975,943720.0,895102.0,964198.0,851907.0,867350.0,0.997268


In [34]:
col_name = [
    "session_id",
    "song_id_0",
    "song_id_1",
    "song_id_2",
    "song_id_3",
    "song_id_4",
]
testX = testX.drop(columns="song_id_0")
testX.columns = col_name

# 5 gen1
n = 5
target = 1
freq = getFreq2(trainX, n, target, 0.1)
result = predict2(testX, freq, n, target)
result = result[[f"top{i+1}" for i in range(target)]]
result_5 = result_5.fillna(result)

# cal 5 2 1 gram
result5 = predict(testX, freq5, n)
testX = merge2freq(result5, freq2)
testX["next_song_id"].fillna(testX["song_id_4"], inplace=True)
testX = merge2freq(testX, freq1)

# combine
result_5["top5"] = result_5["top5"].fillna(testX["next_song_id"])
result_5

Left on: ['song_id_3', 'song_id_4']
Right on: ['song_id', 'next1_song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf38b30>
Left on: ['song_id_3']
Right on: ['song_id']
<generator object merge2freq.<locals>.<genexpr> at 0x7fa50bf38b30>


,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,801419.0,946515.0,660101.0,413130.0,320485.0,NaN
1,1039,229992,252014,810606,388680,82103,388680.0,245916.0,388680.0,388680.0,388680.0,NaN
2,1199,524289,976710,524289,187475,82260,419795.0,82260.0,634769.0,634769.0,79273.0,NaN
3,1489,847611,642783,674238,672342,163516,705778.0,58880.0,905285.0,905285.0,905285.0,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535,1028535,1028535.0,1028535.0,1028535.0,1028535.0,1028535.0,0.988636
143060,317578,467984,610999,140463,508806,352697,352697.0,NaN,NaN,NaN,NaN,NaN
143061,712209,578489,16604,539300,943003,831678,975708.0,997112.0,788164.0,788164.0,788164.0,NaN
143062,713779,720557,735314,790777,708757,862975,943720.0,895102.0,964198.0,851907.0,867350.0,0.997268


In [35]:
# 保留第一個重複值
top15 = result_5[[f"top{i+1}" for i in range(5)]]
top15 = top15.apply(lambda x: x.mask(x.duplicated(keep="first"), np.nan), axis=1)
result_5[["top1", "top2", "top3", "top4", "top5"]] = top15

In [36]:
result_5

,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,801419.0,946515.0,660101.0,413130.0,320485.0,NaN
1,1039,229992,252014,810606,388680,82103,388680.0,245916.0,NaN,NaN,NaN,NaN
2,1199,524289,976710,524289,187475,82260,419795.0,82260.0,634769.0,NaN,79273.0,NaN
3,1489,847611,642783,674238,672342,163516,705778.0,58880.0,905285.0,NaN,NaN,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535,1028535,1028535.0,NaN,NaN,NaN,NaN,0.988636
143060,317578,467984,610999,140463,508806,352697,352697.0,NaN,NaN,NaN,NaN,NaN
143061,712209,578489,16604,539300,943003,831678,975708.0,997112.0,788164.0,NaN,NaN,NaN
143062,713779,720557,735314,790777,708757,862975,943720.0,895102.0,964198.0,851907.0,867350.0,0.997268


In [37]:
# random sample meta_song song_id
def random_sample(series: pd.Series, n: int = 1):
    _ = pd.DataFrame(series)
    random_sample = _.sample(n).reset_index(drop=True)
    return random_sample[0]


# fillna with random sample
sameple_src = meta_song["song_index"].values
sample_series = random_sample(sameple_src, len(result_5))
result_5["top1"] = result_5["top1"].fillna(sample_series)

sample_series = random_sample(sameple_src, len(result_5))
result_5["top2"] = result_5["top2"].fillna(sample_series)

sample_series = random_sample(sameple_src, len(result_5))
result_5["top3"] = result_5["top3"].fillna(sample_series)

sample_series = random_sample(sameple_src, len(result_5))
result_5["top4"] = result_5["top4"].fillna(sample_series)

sample_series = random_sample(sameple_src, len(result_5))
result_5["top5"] = result_5["top5"].fillna(sample_series)

In [38]:
result_5

,session_id,song_id_0,song_id_1,song_id_2,song_id_3,song_id_4,top1,top2,top3,top4,top5,freq
0,598,442405,834888,309466,487545,320485,801419.0,946515.0,660101.0,413130.0,320485.0,NaN
1,1039,229992,252014,810606,388680,82103,388680.0,245916.0,172491.0,585139.0,651026.0,NaN
2,1199,524289,976710,524289,187475,82260,419795.0,82260.0,634769.0,107029.0,79273.0,NaN
3,1489,847611,642783,674238,672342,163516,705778.0,58880.0,905285.0,36719.0,756056.0,NaN
4,1868,742718,856473,935208,798178,736058,710126.0,1026681.0,821055.0,813349.0,996550.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
143059,621339,1028535,1028535,1028535,1028535,1028535,1028535.0,664791.0,975820.0,910627.0,614075.0,0.988636
143060,317578,467984,610999,140463,508806,352697,352697.0,96014.0,634242.0,954174.0,627748.0,NaN
143061,712209,578489,16604,539300,943003,831678,975708.0,997112.0,788164.0,161419.0,743139.0,NaN
143062,713779,720557,735314,790777,708757,862975,943720.0,895102.0,964198.0,851907.0,867350.0,0.997268


In [22]:
N = 3
freq_table = get_freq(train_source, test_source, N)
data = dataX(test_source.copy(), N)
result3 = predict(data, freq_table, N)

In [56]:
N = 4
freq_table = get_freq(train_source, test_source, N)
data = dataX(test_source.copy(), N)
result4 = predict(data, freq_table, N)

In [57]:
N = 5
freq_table = get_freq(train_source, test_source, N)
data = dataX(test_source.copy(), N)
result5 = predict(data, freq_table, N)

In [64]:
N = 6
freq_table = get_freq(train_source, test_source, N)
data = dataX(test_source.copy(), N)
result6 = predict(data, freq_table, N)

In [65]:
N = 7
freq_table = get_freq(train_source, test_source, N)
data = dataX(test_source.copy(), N)
result7 = predict(data, freq_table, N)

In [66]:
N = 8
freq_table = get_freq(train_source, test_source, N)
data = dataX(test_source.copy(), N)
result8 = predict(data, freq_table, N)

In [58]:
# result_tmp = result2.copy()
# result_tmp.fillna(result1, inplace=True)
# result_tmp['next_song_id'].fillna(result_tmp['song_id_1'], inplace=True)
# final_result = result_tmp[['session_id', 'next_song_id', 'next_song_id', 'next_song_id', 'next_song_id', 'next_song_id']]
# final_result.columns = ['session_id', 'top1', 'top2', 'top3', 'top4', 'top5']
# final_result

In [59]:
# result3.fillna(result2, inplace=True)
# # result3.fillna(result1, inplace=True)
# result3['next_song_id'].fillna(result3['song_id_2'], inplace=True)

# final_result = result3[['session_id', 'next_song_id', 'next_song_id', 'next_song_id', 'next_song_id', 'next_song_id']]
# final_result.columns = ['session_id', 'top1', 'top2', 'top3', 'top4', 'top5']
# final_result

In [71]:
result = result_5[["session_id", "top1", "top2", "top3", "top4", "top5"]]
result.head()

,session_id,top1,top2,top3,top4,top5
0,598,801419.0,946515.0,660101.0,413130.0,320485.0
1,1039,388680.0,245916.0,172491.0,585139.0,651026.0
2,1199,419795.0,82260.0,634769.0,107029.0,79273.0
3,1489,705778.0,58880.0,905285.0,36719.0,756056.0
4,1868,710126.0,1026681.0,821055.0,813349.0,996550.0


### Output to csv file

In [191]:
result1['next_song_id'].notna().sum() / len(result1)

0.9999515763119805

In [192]:
result2['next_song_id'].notna().sum() / len(result2)

0.4307822157449686

In [193]:
result3['next_song_id'].notna().sum() / len(result3)

0.12841038760421417

In [150]:
result3.fillna(result2, inplace=True)
result3.fillna(result1, inplace=True)
result3['next_song_id'].notna().sum() / len(result3)

0.9999353439613882

In [194]:
result3['next_song_id'].fillna(result3['song_id_2'], inplace=True)
result3['next_song_id'].notna().sum() / len(result3)

1.0

In [201]:
result3

,session_id,song_id_0,song_id_1,song_id_2,song_id_21,song_id_22,song_id_23,song_id_24,song_id_25,next_song_id,freq
0,1,2ad3043e1a7e459ddb09c5ba27e475f8,7bb8fadfc8f2bf145f4b29a0325fe79a,824c159701c8553b0e38f0d36ddd6197,b186d853cb06ceba2bd56bcdc701b8aa,1071e128ee5f9e2a0f0be0aade025b39,0100af68f3477f6bf724664d0b303f29,22a696cede3609c20ba1bceb8de28032,d6476892c926c88c7f232fd800b75845,824c159701c8553b0e38f0d36ddd6197,NaN
1,2,ae75e2846669037aa91cac098e5009c3,3030938d53f52426ac30f213cf9915a1,6309716e08a58c64871c823f44749686,cd6dade76334f76d993b913d10bb2ac7,822486e25be33911a62b3f12e549d24d,ed37d73c29e3696f65817916316bf05d,197a95a09a55390eb435e35f7dab9f9f,59844ef9a5e469cd13b1208c4f1142d1,6309716e08a58c64871c823f44749686,NaN
2,3,e57c39b5735364b56ed8f743fa948697,e904a955c0350ca5b1b6bb84174ee5be,3b050c1502af19554bb9ef8efc67c00d,a17aab4730e49f8e2972bf26227b890c,868bbeb92bd33a728dcf77d0073cd44d,c4d98c14bd48ec0495239c1a65a5178c,4ea79c3619515a39423851b5ceca965e,d76c868bfb392d89b3ef222907de69fc,3b050c1502af19554bb9ef8efc67c00d,NaN
3,4,a288350a1332f6716cde4eedbbf8443f,6650e87fecc3e969c0d48f7f5e9b2c81,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,NaN
4,5,aaf15c1b9400651add7925c09b609461,c10efc962e6b703053c2b44b7d48e8da,fd96fd2df80abfae763c3693932f8b26,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,b3ac505ffec601de591ce2ce2e010d30,fd96fd2df80abfae763c3693932f8b26,NaN
...,...,...,...,...,...,...,...,...,...,...,...
572254,715317,78c792b3afc368829431ef50a3d871dd,0f59db4259ec62fcf00399b69786a3c0,cb6e848eff5f656c47a4541e6be72079,3c66c29e5c26efc705c261732b977950,a0c8441cbe902320c846da76329a38e6,5d043bbef0ca3cf5385432478f3d3751,3b48c6883b4d16ddc07cb64b7be38d1c,546da46c864f38bb193940a7b8704f44,3c66c29e5c26efc705c261732b977950,1.0
572255,715320,9702af9ea15361bf6b178e073514660d,477ecc7fb10644c0d31408030cd464c6,aa7ce52650c7e4f7979e087cedb89020,b4b1c6c0e7395c50f2cf0702dd1f85d6,dd45bfa90e9186681877c15eb0166a35,c366d245501f007629813a28c9267d11,f4aabaca763955330cdf57f763d2a72c,f4aabaca763955330cdf57f763d2a72c,aa7ce52650c7e4f7979e087cedb89020,NaN
572256,715321,b784c2c425ed31d70f7007e949f6b177,9e65aa27a1c3060b5909dda36859f39f,fba1bc1cf12692b1dd65c98e79985e52,261bceb1e3440067b85f5c54444d91be,65961a80125282df5c95b4c0a3606cc5,a91ec5b2b51524c53ce866f621208087,ab8683d279c095064693c282b8353b98,01c48d6a895a6efed0d5afba83b87902,fba1bc1cf12692b1dd65c98e79985e52,NaN
572257,715322,826cf1a6e383fdbd900c3865d2edb010,940ed872f8ae21d392495623c29df127,ceee2a16d7fb7531cdc633ac478fe46a,c31a95cdd5635c6edfbde9143286b9b7,aa41008b812435306a6fe831f40b0c11,d1492ba4a2fdf555510934e9740ea8c3,f1bc21ff88fe47bac896fbe591fa2f5f,d1492ba4a2fdf555510934e9740ea8c3,c31a95cdd5635c6edfbde9143286b9b7,1.0


In [198]:
final_result = result3[['session_id', 'next_song_id', 'next_song_id', 'next_song_id', 'next_song_id', 'next_song_id']]
final_result.columns = ['session_id', 'top1', 'top2', 'top3', 'top4', 'top5']
final_result

,session_id,top1,top2,top3,top4,top5
0,1,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197
1,2,6309716e08a58c64871c823f44749686,6309716e08a58c64871c823f44749686,6309716e08a58c64871c823f44749686,6309716e08a58c64871c823f44749686,6309716e08a58c64871c823f44749686
2,3,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d
3,4,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3
4,5,fd96fd2df80abfae763c3693932f8b26,fd96fd2df80abfae763c3693932f8b26,fd96fd2df80abfae763c3693932f8b26,fd96fd2df80abfae763c3693932f8b26,fd96fd2df80abfae763c3693932f8b26
...,...,...,...,...,...,...
572254,715317,3c66c29e5c26efc705c261732b977950,3c66c29e5c26efc705c261732b977950,3c66c29e5c26efc705c261732b977950,3c66c29e5c26efc705c261732b977950,3c66c29e5c26efc705c261732b977950
572255,715320,aa7ce52650c7e4f7979e087cedb89020,aa7ce52650c7e4f7979e087cedb89020,aa7ce52650c7e4f7979e087cedb89020,aa7ce52650c7e4f7979e087cedb89020,aa7ce52650c7e4f7979e087cedb89020
572256,715321,fba1bc1cf12692b1dd65c98e79985e52,fba1bc1cf12692b1dd65c98e79985e52,fba1bc1cf12692b1dd65c98e79985e52,fba1bc1cf12692b1dd65c98e79985e52,fba1bc1cf12692b1dd65c98e79985e52
572257,715322,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7


In [202]:
def songindex2songid(df):
    _df = df.copy()
    count = _df.shape[1]
    for i in range(1, 6):
        _df = _df.rename(columns={f"top{i}": "song_index"})
        _df = _df.merge(
            meta_song[["song_index", "song_id"]], on="song_index", how="left"
        )
        _df.rename(columns={"song_id": f"top{i}"}, inplace=True)
        _df.drop(columns=["song_index"], inplace=True)

    return _df

In [41]:
res = songindex2songid(result)

In [203]:
res.to_csv("results/result.csv", index=False)